In [ ]:
# Sklearn scaling
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
import cv2
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image

In [ ]:
# Set up file and dataframe to store accuracy and loss of modelfit epochs
# The code in this cell was commented out after creating the dataframe for all 8 optimizers

# file = "modelfit_history.csv"
# modelfit_hist_df = pd.read_csv(file)
# modelfit_hist_df

### Load and process image data using CV2

- Data is found in the kaggle zip file https://www.kaggle.com/kishanj/music-notes-datasets
- Zip file was extracted.
- All of the image files were in folders for by note type (Whole, Half, Quarter, Eight, Sixteenth)
- In order to simplify the code, all files were copied out of their subdirectories into single "Images" folder. 
- The Images folder is part of gitignore, and therefore unavailable from our git repo

In [ ]:
# Loop through the Images folder and process each image file
# Append the image to a list and append the class name to a separate list

img_data_array=[]
class_name=[]
folder_name = "Images"

# This code based on https://towardsdatascience.com/loading-custom-image-dataset-for-deep-learning-models-part-1-d64fa7aaeca6
# This is the code that required us to install OpenCV2
# This code also handles the scaling needed

for file in os.listdir(os.path.join(folder_name)):
    
    image_path= os.path.join(folder_name,  file)
    image= cv2.imread( image_path, cv2.IMREAD_GRAYSCALE)
    #image=np.array(image)  ????

    image = image.astype('float32')
    image /= 255 #this gets black and white to 1s and 0s 
    img_data_array.append(image)
    note_class = file[0:1]

    class_name.append(note_class)


In [ ]:
def create_dataset(notes_folder):
    class_name = []
    img_data_array = []
    
    for note_dir in os.listdir(notes_folder):
        
        for file in os.listdir(os.path.join(notes_folder, note_dir)):
            image = os.path.join(notes_folder, note_dir, file)
            image = cv2.imread(notes_folder, cv2.COLOR_BGR2RGB)
            #image = cv2.resize( image, (IMG_HEIGHT, IMG_WIDTH), interpolation = cv2.INTER_AREA)
            #image = np.array(image)  # converts the image to a numpy array
            image = image.astype('float32')
            image /= 255
            img_data_array.append(image)
            class_name.append(note_dir)
            
    print(img_data_array[0])
    return img_data_array, class_name

In [ ]:
# Assign a unique number to each class

note_classifier_dict = {key:value for value, key in enumerate(np.unique(class_name))}

note_classifier_dict

In [ ]:
# This is y, converting the class names to a numeric value for all values

target_val = [note_classifier_dict[class_name[i]] for i in range(len(class_name))]

# target_val

In [ ]:
# Examine the image array ... setting the threshold to 4096 or above avoids truncating the array

np.set_printoptions(threshold=4096)

img_data_array[0]

In [ ]:
# Test to make sure our images are still good

import matplotlib.pyplot as plt
plt.imshow(img_data_array[10], cmap=plt.cm.Greys)

In [ ]:
# Flatten the 64x64 pixels to 1D array 
# This is similar to what was done in MNIST class example

num_dimensions = 64*64

In [ ]:
# Make this an np array so we can use reshape on our array to flatten
# the array from 64 x 64 to 1 x 4096 (from a vector matrix to a scalar matrix)

x=np.array(img_data_array, np.float32)

reshape_test = x.reshape(x.shape[0],num_dimensions)

In [ ]:
reshape_test.shape

### Split data into train and test sets

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(reshape_test,target_val, random_state=42)

### Scaling and Normalization

In [ ]:
# *NOTE*:  Instead of using the code in this cell, we performed the scaling manually above 
# using 'image /= 255' as we loaded each image 

# Next, we normalize our training data to be between 0 and 1
# scaler = MinMaxScaler().fit(X_train)

# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

# X_train[0]

### One-Hot Encoding

In [ ]:
# Our Training and Testing labels are integer encoded from 0 to 4
y_train[:20]

In [ ]:
# We have five categories: whole, half, quarter, eighth, and sixteenth notes
num_classes = 5

# Encode the target using one-hot encoding
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_train[0]

## Building our Model


### Create a Sequential model

In [ ]:
model = Sequential()

In [ ]:
# Add the first layer where the input dimensions are 4096 pixels
# Activation function using 'relu' for the hidden layers and 'softmax' for the output layer
# Each of the hidden layers are densely connected and have 100 nodes per layer
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))


### Add a second hidden layer with 100 densely connected nodes

In [ ]:
model.add(Dense(100, activation='relu'))

### Final output layer uses softmax activation function for logistic regression

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

### Model Summary

In [ ]:
model.summary()

### Compile and Train our Model 

These were run one at a time, with only one optimizer in play for each run
All of the optimizers were run.  

In [ ]:
# optimizer = "Adam"
# optimizer = "SGD"
optimizer = "RMSprop"
# optimizer = "Adadelta"
# optimizer = "Adagrad"
# optimizer = "Adamax"
# optimizer = "Nadam"
#optimizer = tf.keras.optimizers.Ftrl(learning_rate=1e-5)

In [ ]:
model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Train (fit) our model using the training data

We are setting the model.fit() equal to a variable 'history' in order to use it to create the plots

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=0
    )

In [ ]:
#below 3 cells were just to store model fit timings to dataframe. commented out after timings achieved.
# import time

In [ ]:
# start = time.time()
# history = model.fit(
#     X_train,
#     y_train,
#     epochs=100,
#     shuffle=True,
#     verbose=0
#     )
# end = time.time()
# print(end - start)

In [ ]:
#timer_list = []
# timer_list.append([optimizer, round(end - start,2)])
# timer_list

In [ ]:
# timer_df=pd.DataFrame(timer_list,columns=["Optimizer","Model_Fit_Time"])
# timer_df.to_csv('Model_fit_100Epoch_timer.csv',index=False)

In [ ]:
#Build a dataframe to hold the model fit history values
modelfit_hist_df=pd.DataFrame()   #only needed for first run
modelfit_hist_df["ftrl_loss"] = history.history['loss']  #only needed for last one
modelfit_hist_df["ftrl_acc"] = history.history['acc']   #only needed for last one

In [ ]:
# modelfit_hist_df[optimizer+"_loss"] = history.history['loss']
# modelfit_hist_df[optimizer+"_acc"] = history.history['acc']

In [ ]:
# modelfit_hist_df #test output

In [ ]:
# modelfit_hist_df.to_csv(file, index=False)
#modelfit_hist_df.to_csv('FTRL_ACC_LOSS.csv', index=False)

In [ ]:
#history.model #testing output

### Visualization of the Optimizer performance
The loss and accuracy for each optimizer and save each plot as a file and save the model

In [ ]:
plt.plot(history.history['loss'], label='Loss (training data)')
# plt.plot(history.history['acc'], label='MAE (validation data)')
plt.plot(history.history['acc'], label='Accuracy (training data)')
#plt.title('Loss and Accuracy vs. Epochs - Optimizer: '+ optimizer)
plt.title('Loss and Accuracy vs. Epochs - Optimizer: FTRL')
plt.ylabel('Model Efficiency')
plt.xlabel('Epochs')
plt.legend(loc="middle")
#plt.savefig("Results/100epoch_relu_" + optimizer + ".jpeg")
plt.savefig("Results/100epoch_relu_FTRL.jpeg")
plt.show()

In [ ]:
# commented out so we don't resave over existing file
#model.save("Results/Notes_relu_" + optimizer + ".h5")

## Evaluating the Model

Evaluate each of the 8 optimizers and store the loss and accuracy in a new DF

In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
#define list to hold model eval results
df_list = []


In [ ]:
model_loss, model_accuracy = model.evaluate(X_test,y_test, verbose=2)

In [ ]:
def model_predictions(results_folder):

    
     for model_file in os.listdir(os.path.join(results_folder)):
            if "h5" in model_file:
                print("working on file: "+model_file)
                model = load_model(os.path.join(results_folder,model_file))
#                 break #for testing purposes only
                try:
                    model_loss, model_accuracy = model.evaluate(X_test,y_test, verbose=2)
                    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
                    df_list.append([model_file, model_loss,model_accuracy])
                except:
                    print("Ftrl model defies being saved and will not load. filename:"+model_file)
                
                
                

In [ ]:
model_predictions('Results')

In [ ]:
# put results to df
print(df_list)
results_df = pd.DataFrame(df_list,columns=["Model","Loss","Accuracy"])

In [ ]:
#save to csv for later loading/graphing elsewhere
results_df.to_csv(os.path.join('Results','Model_predict_loss_acc.csv'),index=False)

# Try our model on some hand drawn notes
shout out to MS Paint

In [ ]:
model = load_model(os.path.join('Results','notes_relu_Adam.h5'))  # change file for the model you want to try

In [ ]:
#put a file in the TestImage folder and set it to filepath here.  Note we've tested a few different files already:
#filepath = "TestImage/note_pensketch.jpg"   # so it thought this guy was an eigth note
#filepath = "TestImage/mn5.png"  # successfully predicted Fukhrudin's whole notes
#filepath = "TestImage/sharon.png"  
filepath = "TestImage/eigth_fukhrudin.png"  

In [ ]:
from tensorflow.keras.preprocessing import image
image_size = (64, 64)
im = image.load_img(filepath, target_size=image_size, color_mode="grayscale")
im

In [ ]:
# Convert the image to a numpy array 
from tensorflow.keras.preprocessing.image import img_to_array
image = img_to_array(im)
image.shape

In [ ]:
# Scale the image pixels by 255 (or use a scaler from sklearn here)
image /= 255

# Flatten into a 1x28*28 array 
img = image.flatten().reshape(-1, 64*64)
img.shape

In [ ]:
plt.imshow(img.reshape(64, 64), cmap=plt.cm.Greys)

dictionary of classifications
{'eigth': 0, 'half': 1, 'quarter': 2, 'sixteenth': 3, 'whole': 4}

In [ ]:
model.predict_classes(img)
